In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

from statsmodels.formula.api import mixedlm
from scipy.stats import wilcoxon, ttest_rel
from itertools import combinations

from scipy.stats import ConstantInputWarning
import warnings
warnings.filterwarnings("ignore", category=ConstantInputWarning)

In [2]:
def detect_factors_and_metrics(df):
    """
    Clean a sheet:
      - drop all-NaN rows
      - strip column names
      - forward-fill factor columns (KG, Model, Complexity)
      - cast factors to categorical
      - detect numeric metric columns automatically
    """
    df = df.dropna(how="all").copy()
    df.columns = [str(c).strip() for c in df.columns]

    factor_candidates = ["KG", "Model", "Complexity"]
    factors = [c for c in factor_candidates if c in df.columns]

    # Forward-fill factor columns (your sheets have KG/Model only on first row of a block)
    for f in factors:
        df[f] = df[f].ffill()
        df[f] = df[f].astype("category")

    # Metrics = numeric columns that are not factors and not totally NaN
    metric_cols = []
    for c in df.columns:
        if c in factors:
            continue
        if pd.api.types.is_numeric_dtype(df[c]) and df[c].notna().any():
            metric_cols.append(c)

    return df, factors, metric_cols

In [3]:
def run_anova_and_posthoc(df, factors, metric, sheet_name):
    """Run ANOVA, Tukey and Wilcoxon tests for one metric on one sheet."""
    data = df.dropna(subset=[metric]).copy()

    print("\n" + "=" * 120)
    print(f"SHEET: {sheet_name} | METRIC: {metric}")
    print("=" * 120)

    # If metric is constant, no point in testing anything
    if data[metric].nunique() < 2:
        print("Metric is constant across rows; skipping statistical tests.")
        return

    has_kg = "KG" in factors
    has_complexity = "Complexity" in factors
    has_model = "Model" in factors

    # -------------------- ANOVA / mixed model --------------------
    anova_done = False

    if has_kg or has_complexity:
        try:
            # Build formula depending on which factors exist
            if has_kg and has_complexity:
                # 2-way with interaction, plus Model as a blocking factor if present
                formula = f"{metric} ~ C(KG) * C(Complexity)"
                if has_model:
                    formula += " + C(Model)"
            elif has_kg:
                formula = f"{metric} ~ C(KG)"
                if has_model:
                    formula += " + C(Model)"
            else:  # only complexity
                formula = f"{metric} ~ C(Complexity)"
                if has_model:
                    formula += " + C(Model)"

            print(f"Attempting ANOVA with formula: {formula}")
            model = smf.ols(formula, data=data).fit()
            anova_table = anova_lm(model, typ=2)

            # Add eta-squared effect size
            ss_total = anova_table["sum_sq"].sum()
            anova_table["eta_sq"] = anova_table["sum_sq"] / ss_total

            print("\nANOVA table:")
            print(anova_table)
            anova_done = True

        except Exception as e:
            print("\n⚠️ ANOVA failed, attempting mixed-effects model instead.")
            print(f"Reason: {e}")

            # Mixed model: KG as fixed effect, Model as random effect (if both present)
            if has_kg and has_model:
                try:
                    print("\nMixed-effects model: metric ~ KG with random intercept per Model")
                    mx = mixedlm(f"{metric} ~ C(KG)", data=data, groups=data["Model"])
                    res = mx.fit()
                    print(res.summary())
                except Exception as e2:
                    print("Mixed-effects model also failed:", e2)
            else:
                print("No appropriate structure for mixed-effects model; skipping.")

    else:
        print("No KG or Complexity columns found; skipping ANOVA/mixed.")

    # -------------------- Tukey HSD: KG --------------------
    if has_kg and data["KG"].nunique() > 1:
        try:
            print("\nPost-hoc: KG (Tukey HSD)")
            tuk_kg = pairwise_tukeyhsd(endog=data[metric],
                                       groups=data["KG"],
                                       alpha=0.05)
            print(tuk_kg)
        except Exception as e:
            print("Tukey for KG failed:", e)

    # -------------------- Tukey HSD: Complexity --------------------
    if has_complexity and data["Complexity"].nunique() > 1:
        try:
            print("\nPost-hoc: Complexity (Tukey HSD)")
            tuk_c = pairwise_tukeyhsd(endog=data[metric],
                                      groups=data["Complexity"],
                                      alpha=0.05)
            print(tuk_c)
        except Exception as e:
            print("Tukey for Complexity failed:", e)

        # KG × Complexity combos
        if has_kg:
            try:
                print("\nPost-hoc: KG × Complexity combinations (Tukey HSD)")
                combos = data["KG"].astype(str) + "_" + data["Complexity"].astype(str)
                tuk_kc = pairwise_tukeyhsd(endog=data[metric],
                                           groups=combos,
                                           alpha=0.05)
                print(tuk_kc)
            except Exception as e:
                print("Tukey for KG×Complexity combos failed:", e)

    # -------------------- Wilcoxon: KG (paired across models) --------------------
    # Here we treat each model as a subject, compare Big vs Small (or two KG levels)
    if has_kg and has_model:
        wide_kg = data.pivot_table(index="Model", columns="KG", values=metric)

        if wide_kg.shape[1] >= 2:
            # If more than 2 levels (not in your current file, but just in case),
            # compare them pairwise.
            levels = list(wide_kg.columns)
            print("\nWilcoxon signed-rank tests for KG (pairwise across models):")
            for a, b in combinations(levels, 2):
                sub = wide_kg[[a, b]].dropna()
                if len(sub) > 1:
                    try:
                        w_stat, w_p = wilcoxon(sub[a], sub[b])
                        print(f"  {a} vs {b}: n = {len(sub)}, W = {w_stat:.4f}, p = {w_p:.4g}")
                        # Optional: paired t-test for reference
                        t_stat, t_p = ttest_rel(sub[a], sub[b])
                        print(f"    (paired t-test: t = {t_stat:.4f}, p = {t_p:.4g})")
                    except Exception as e:
                        print(f"  {a} vs {b}: Wilcoxon failed:", e)
                else:
                    print(f"  {a} vs {b}: not enough paired data, skipped.")
        else:
            print("\nWilcoxon KG: skipped (KG does not have at least 2 levels).")

    # -------------------- Wilcoxon: Complexity (pairwise across models) --------------------
    if has_complexity and has_model:
        wide_c = data.pivot_table(index="Model",
                                  columns="Complexity",
                                  values=metric)

        levels = [lvl for lvl in wide_c.columns if wide_c[lvl].notna().any()]
        if len(levels) >= 2:
            print("\nWilcoxon signed-rank tests for Complexity (pairwise across models):")
            for a, b in combinations(levels, 2):
                sub = wide_c[[a, b]].dropna()
                if len(sub) > 1:
                    try:
                        w_stat, w_p = wilcoxon(sub[a], sub[b])
                        print(f"  {a} vs {b}: n = {len(sub)}, W = {w_stat:.4f}, p = {w_p:.4g}")
                        # Optional: paired t-test for reference
                        t_stat, t_p = ttest_rel(sub[a], sub[b])
                        print(f"    (paired t-test: t = {t_stat:.4f}, p = {t_p:.4g})")
                    except Exception as e:
                        print(f"  {a} vs {b}: Wilcoxon failed:", e)
                else:
                    print(f"  {a} vs {b}: not enough paired data, skipped.")
        else:
            print("\nWilcoxon Complexity: skipped (fewer than 2 complexity levels with data).")

In [10]:
def analyze_excel(file_path):
    xls = pd.ExcelFile(file_path)
    print(f"Found sheets: {xls.sheet_names}")
    

    # for sheet in xls.sheet_names:
    for sheet in ['H1-AM&C']:
        print("\n" + "#" * 120)
        print(f"ANALYZING SHEET: {sheet}")
        print("#" * 120)

        df_raw = xls.parse(sheet)
        df, factors, metric_cols = detect_factors_and_metrics(df_raw)

        print(f"Detected factor columns: {factors}")
        print(f"Detected metric columns: {metric_cols}")

        if not metric_cols:
            print("No numeric metrics detected, skipping sheet.")
            continue

        for metric in metric_cols:
            run_anova_and_posthoc(df, factors, metric, sheet)

In [ ]:
# ---------------- CONFIG ----------------
EXCEL_FILE = "all_models_consolidated_results.xlsx" 
# ----------------------------------------

In [12]:
analyze_excel(EXCEL_FILE)

Found sheets: ['Screen', 'All_Screen', 'Screen with C', 'H1-AM', 'H1-AM&C', 'H2-AM', 'H2-AM&C']

########################################################################################################################
ANALYZING SHEET: H1-AM&C
########################################################################################################################
Detected factor columns: ['KG', 'Model', 'Complexity']
Detected metric columns: ['rows', 'vars', 'latency_p50_ms', 'latency_p95_ms', 'latency_mean_ms', 'lexical_query_overlap', 'semantic_similarity_to_CQ', 'semantic_soft_coverage_to_CQ', 'tuple_cohesion', 'determinism_score', 'satisfiability_binding_score', 'h1_overall', 'semantic_diversity_score', 'syntax_ok_rate', 'satisfiable_rate', 'deterministic_rate']

SHEET: H1-AM&C | METRIC: rows
Attempting ANOVA with formula: rows ~ C(KG) * C(Complexity) + C(Model)

ANOVA table:
                           sum_sq    df         F    PR(>F)    eta_sq
C(KG)                3.846991e+07   1.0

/tmp/ipykernel_2035519/1104738204.py:106: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  wide_kg = data.pivot_table(index="Model", columns="KG", values=metric)
/tmp/ipykernel_2035519/1104738204.py:131: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  wide_c = data.pivot_table(index="Model",


  Multiple Comparison of Means - Tukey HSD, FWER=0.05   
 group1   group2  meandiff p-adj   lower   upper  reject
--------------------------------------------------------
 Complex Moderate  -1.5265 0.0258 -2.8844 -0.1687   True
 Complex   Simple  -1.1795 0.0964 -2.5373  0.1783  False
Moderate   Simple   0.3471 0.7976 -1.0108  1.7049  False
--------------------------------------------------------

Post-hoc: KG × Complexity combinations (Tukey HSD)
        Multiple Comparison of Means - Tukey HSD, FWER=0.05        
    group1         group2     meandiff p-adj   lower  upper  reject
-------------------------------------------------------------------
   Big_Complex   Big_Moderate  -1.9608 0.1895 -4.4986  0.577  False
   Big_Complex     Big_Simple   -1.749 0.2892 -4.2868 0.7888  False
   Big_Complex  Small_Complex  -0.6453 0.9623 -3.1831 1.8925  False
   Big_Complex Small_Moderate  -1.7376 0.2955 -4.2754 0.8002  False
   Big_Complex   Small_Simple  -1.2553 0.6256 -3.7931 1.2825  False
  Big

/tmp/ipykernel_2035519/1104738204.py:106: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  wide_kg = data.pivot_table(index="Model", columns="KG", values=metric)
/tmp/ipykernel_2035519/1104738204.py:131: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  wide_c = data.pivot_table(index="Model",


     Multiple Comparison of Means - Tukey HSD, FWER=0.05      
 group1   group2   meandiff p-adj    lower      upper   reject
--------------------------------------------------------------
 Complex Moderate    5.8034    1.0 -1646.3449 1657.9517  False
 Complex   Simple -345.8477 0.8587  -1997.996 1306.3006  False
Moderate   Simple -351.6511 0.8544 -2003.7994 1300.4972  False
--------------------------------------------------------------

Post-hoc: KG × Complexity combinations (Tukey HSD)
            Multiple Comparison of Means - Tukey HSD, FWER=0.05            
    group1         group2      meandiff  p-adj    lower      upper   reject
---------------------------------------------------------------------------
   Big_Complex   Big_Moderate  -128.8981    1.0 -3089.5754 2831.7791  False
   Big_Complex     Big_Simple   525.9198 0.9922 -2434.7574 3486.5971  False
   Big_Complex  Small_Complex   598.6767  0.986 -2362.0005 3559.3539  False
   Big_Complex Small_Moderate   739.1817 0.9651 -22

...

/tmp/ipykernel_2035519/1104738204.py:106: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  wide_kg = data.pivot_table(index="Model", columns="KG", values=metric)
/tmp/ipykernel_2035519/1104738204.py:131: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  wide_c = data.pivot_table(index="Model",


  Multiple Comparison of Means - Tukey HSD, FWER=0.05  
 group1   group2  meandiff p-adj   lower  upper  reject
-------------------------------------------------------
 Complex Moderate   -0.008 0.9986 -0.4043 0.3884  False
 Complex   Simple   0.1208  0.726 -0.2755 0.5172  False
Moderate   Simple   0.1288 0.6956 -0.2676 0.5251  False
-------------------------------------------------------

Post-hoc: KG × Complexity combinations (Tukey HSD)
        Multiple Comparison of Means - Tukey HSD, FWER=0.05        
    group1         group2     meandiff p-adj   lower  upper  reject
-------------------------------------------------------------------
   Big_Complex   Big_Moderate  -0.0818 0.9992 -0.8289 0.6652  False
   Big_Complex     Big_Simple   -0.025    1.0 -0.7721 0.7221  False
   Big_Complex  Small_Complex   -0.125 0.9941 -0.8721 0.6221  False
   Big_Complex Small_Moderate  -0.0591 0.9998 -0.8061  0.688  False
   Big_Complex   Small_Simple   0.1417 0.9895 -0.6054 0.8887  False
  Big_Modera

/tmp/ipykernel_2035519/1104738204.py:106: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  wide_kg = data.pivot_table(index="Model", columns="KG", values=metric)
/root/.local/lib/python3.10/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/root/.local/lib/python3.10/site-packages/scipy/stats/_morestats.py:4102: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/tmp/ipykernel_2035519/1104738204.py:131: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and

  Multiple Comparison of Means - Tukey HSD, FWER=0.05  
 group1   group2  meandiff p-adj   lower  upper  reject
-------------------------------------------------------
 Complex Moderate   0.0148 0.9945 -0.3591 0.3886  False
 Complex   Simple      0.1 0.7808 -0.2739 0.4739  False
Moderate   Simple   0.0852  0.835 -0.2886 0.4591  False
-------------------------------------------------------

Post-hoc: KG × Complexity combinations (Tukey HSD)
        Multiple Comparison of Means - Tukey HSD, FWER=0.05        
    group1         group2     meandiff p-adj   lower  upper  reject
-------------------------------------------------------------------
   Big_Complex   Big_Moderate  -0.0659 0.9996 -0.7723 0.6404  False
   Big_Complex     Big_Simple    0.025    1.0 -0.6814 0.7314  False
   Big_Complex  Small_Complex   -0.025    1.0 -0.7314 0.6814  False
   Big_Complex Small_Moderate   0.0705 0.9995 -0.6359 0.7768  False
   Big_Complex   Small_Simple     0.15 0.9826 -0.5564 0.8564  False
  Big_Modera

/tmp/ipykernel_2035519/1104738204.py:106: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  wide_kg = data.pivot_table(index="Model", columns="KG", values=metric)
/tmp/ipykernel_2035519/1104738204.py:131: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  wide_c = data.pivot_table(index="Model",
